In [1]:
import cv2
import numpy as np

In [2]:
# 최소 정확도
min_confidence = 0.9

In [3]:
# yolo 모델을 로딩한다.
net = cv2.dnn.readNet('opencv_data/yolo/yolov3.weights', 'opencv_data/yolo/yolov3.cfg')

In [4]:
# 물체 종류를 리스트로 추출한다.
with open('opencv_data/yolo/coco.names', 'r') as fp:
    classes = [line.strip() for line in fp.readlines()]
    # print(classes)

In [5]:
# 은닉층의 이름들을 추출한다.
layer_names = net.getLayerNames()

# 다수의 신경망을 사용하는 yolo  모델에서 각 신경망의 출력층 이름을 가져온다.
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [6]:
# 각 물체에 대해 표시할 영역의 색상을 랜덤으로 정해준다,
colors = np.random.uniform(0, 255, size=(len(classes),  2))

In [7]:
# 이미지를 불러온다.
# img = cv2.imread('opencv_data/image/yolo_01.jpg')
img = cv2.imread('opencv_data/image/marathon_02.jpg')

In [8]:
# 이미지 정보를 추출한다.
height, width, channels = img.shape

# 원본 이미지를 띄운다.
cv2.imshow('Orinigal',img)

In [9]:
# 이미지를 리사이징해서 2진데이터로 바꾼다.
# (320x320) 크기를 작게하면 정확도는 떨어지지만 속도는 올라간다.
blob = cv2.dnn.blobFromImage(img, 0.00382,(416, 416), (0,0,0), True, crop=False)

In [10]:
# 2진 데이터를 넣어서 예측 결과를 가져온다.
net.setInput(blob)
outs = net.forward(output_layers)
#print(outs)

In [11]:
# 예측된 결과를 담을 리스트
# 인지된 사물의 인덱스 번호
class_ids = []
# 예측 정확도
confidences = []
# 인지된 사물의 영역
boxes = []

In [12]:
# 각 신경망이 예측한 결과 묶음의 개수(3개)만큼 반복한다.
for out in outs:
    # 예측된 사물의 수만큼 반복한다.
    for detection in out:
        
        # 예측 정확도를 가지고 있는 배열을 추출한다.
        scores = detection[5:]
        
        # 가장 높은 정확도를 가지고 있는 곳의 인덱스를 추출한다.
        class_id = np.argmax(scores)
        # print(class_id)
        # print(collections.Counter(scores))
        
        # 예측 정확도를 가져온다.
        confidence = scores[class_id]
        # print(confidence)
        
        # 예측 정확도가 설정한 최소 정확도보다 큰것만 사용한다.
        if confidence > min_confidence:
            # 감지한 물체에 영역을 표시한다.
            # 물체의 중앙점 x,y 좌표
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)

            # 물체의 가로 세로 길이
            w = int(detection[2] * width)
            h = int(detection[3] * height)

            # 영역의 x,y 좌표
            x = int(center_x - w/2)
            y = int(center_y - h/2)

            # print(x,y,w,h)

            cv2.rectangle(img,(x,y), (x+w, y+h), colors[class_id],2)
            
            # 물체 이름을 표시한다.
            # 물체이름과 정확도 표시
            text = f'{classes[class_id]} : {int(confidence * 100)}'
            cv2.putText(img, text, (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, colors[class_id],1)
            
            # 검출된 항목의 인덱스를 담는다.
            class_ids.append(class_id)
            # 검출된 항목의 예측 정확도를 담는다.
            confidences.append(confidence)
            # 인지된 사물의 영역을 담는다.
            boxes.append((x,y,w,h))

In [13]:
# 결과 출력
for idx in range(len(class_ids)):
    a1 = classes[class_ids[idx]]
    a2 = confidences[idx]
    a3 = boxes[idx]
    
    print(f'이름:{a1}')
    print(f'정확도:{a2}')
    print(f'영역 : {a3}')
    print('-'*20)

이름:person
정확도:0.9992872476577759
영역 : (-5, 10, 228, 186)
--------------------
이름:person
정확도:0.9999347925186157
영역 : (6, 10, 231, 185)
--------------------
이름:person
정확도:0.9856717586517334
영역 : (28, 7, 214, 190)
--------------------
이름:tie
정확도:0.9214240312576294
영역 : (110, 194, 50, 10)
--------------------


In [14]:
cv2.imshow('YOLO',img)
cv2.waitKey()
cv2.destroyAllWindows()

haar 
=> [좌측상단 x, 좌측상단 y, 가로 w, 세로 h]

dnn 
=> [좌측상단 x비율, 좌측상단 y비율, 우측하단 x비율, 우측하단 y비율]

yolo
=> [좌측상단 x비율, 좌측상단 y비율, 가로 비율, 세로 비율]